In [1]:
import numpy as np
import pandas as pd


In [5]:
df=pd.read_csv('CleanedTravel.csv',index_col=[0])
df.head()

,ProdTaken,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,Designation,MonthlyIncome,TotalVisiting
0,1,41.0,Self Enquiry,3,6.0,Salaried,Female,3.0,Deluxe,3.0,Unmarried,1.0,1,2,1,Manager,20993.0,3.0
1,0,49.0,Company Invited,1,14.0,Salaried,Male,4.0,Deluxe,4.0,Divorced,2.0,0,3,1,Manager,20130.0,5.0
2,1,37.0,Self Enquiry,1,8.0,Free Lancer,Male,4.0,Basic,3.0,Unmarried,7.0,1,3,0,Executive,17090.0,3.0
3,0,33.0,Company Invited,1,9.0,Salaried,Female,3.0,Basic,3.0,Divorced,2.0,1,5,1,Executive,17909.0,3.0
4,0,36.0,Self Enquiry,1,8.0,Small Business,Male,3.0,Basic,4.0,Divorced,1.0,0,5,1,Executive,18468.0,2.0


In [6]:
x=df.drop('ProdTaken',axis=1)
y=df['ProdTaken']

In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=42)

In [9]:
cat_features = x.select_dtypes(include="object").columns
print(cat_features)

Index(['TypeofContact', 'Occupation', 'Gender', 'ProductPitched',
       'MaritalStatus', 'Designation'],
      dtype='object')


In [10]:
from sklearn.preprocessing import OneHotEncoder
encoder=OneHotEncoder(drop='first')
encoded=encoder.fit_transform(x_train[cat_features]).toarray()
one_hot_df=pd.DataFrame(encoded,columns=encoder.get_feature_names_out(cat_features),index=x_train.index)
x_train=pd.concat([x_train,one_hot_df],axis=1).drop(columns=cat_features)


In [11]:

encoded=encoder.transform(x_test[cat_features]).toarray()
one_hot_df=pd.DataFrame(encoded,columns=encoder.get_feature_names_out(cat_features),index=x_test.index)
x_test=pd.concat([x_test,one_hot_df],axis=1).drop(columns=cat_features)


In [13]:
! pip install xgboost

In [23]:
from xgboost import XGBClassifier
model=XGBClassifier(tree_method="hist", device="cuda",n_estimators=2, max_depth=2, learning_rate=1, objective='binary:logistic')
model.fit(x_train,y_train)
y_pred=model.predict(x_test)

In [24]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.88      0.91      0.90       999
           1       0.53      0.42      0.47       223

    accuracy                           0.82      1222
   macro avg       0.70      0.67      0.68      1222
weighted avg       0.81      0.82      0.82      1222



In [25]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1, 0.01],
    'n_estimators': [50, 100],
}

grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='accuracy', verbose=1)
grid.fit(x_train, y_train)

print("Best parameters:", grid.best_params_)
print("Best score:", grid.best_score_)

Fitting 3 folds for each of 12 candidates, totalling 36 fits
Best parameters: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100}
Best score: 0.8857064920894708
